In [ ]:
#WRDS login details are as follows:
    
#user: muhdnoor
#password: WRDSaccess_135@

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
# Shrinking the cross-section (2020) | S Kozak, S Nagel, S Santosh

# Summary: The paper constructs a robust stochastic discount factor (SDF) 
# summarizing the joint explanatory power of a large number of cross-sectional stock 
# return predictors. Two sets of independent characteristics are constructed, namely 
# 50 anomaly characteristics and 70 WRDS Industry Financial Ratios (WFR).

In [ ]:
import wrds
import pandas as pd
import numpy as np
import json
import os
from pandas.tseries.offsets import *

# Load config file
with open('config.json') as config_file:
    data = json.load(config_file)

start_year = data['start_year']
end_year = data['end_year']
wrds_username = data['wrds_username']

parm = {"start_year": start_year, "end_year": end_year}

In [ ]:
# Query data from WRDS
conn = wrds.Connection(wrds_username=wrds_username)

crsp_msf = conn.raw_sql("""
                      select a.ret, a.retx, a.prc, a.shrout, a.vol, a.date, a.permno
                      from crsp.msf as a
                      left join crsp.msenames as b
                      on a.permno=b.permno
                      where b.namedt<=a.date
                      and a.date<=b.nameendt
                      and a.date >= '01/01/%(start_year)s'
                      and a.date <= '12/31/%(end_year)s'
                      and b.exchcd between 1 and 3
                      """, params=parm)

conn.close()

In [ ]:
# Query data from WRDS
conn = wrds.Connection(wrds_username=wrds_username)

comp_annual = conn.raw_sql("""
                    /*header info*/
                    select c.gvkey, f.cusip, f.datadate, f.fyear, c.cik, substr(c.sic,1,2) as sic2, c.sic, c.naics,

                    /*firm variables*/
                    /*income statement*/
                    f.sale, f.revt, f.cogs, f.xsga, f.dp, f.xrd, f.xad, f.ib, f.ebitda,
                    f.ebit, f.nopi, f.spi, f.pi, f.txp, f.ni, f.txfed, f.txfo, f.txt, f.xint,

                    /*CF statement and others*/
                    f.capx, f.oancf, f.dvt, f.ob, f.gdwlia, f.gdwlip, f.gwo, f.mib, f.oiadp, f.ivao, f.prstkc,

                    /*assets*/
                    f.rect, f.act, f.che, f.ppegt, f.invt, f.at, f.aco, f.intan, f.ao, f.ppent, f.gdwl, f.fatb, f.fatl,

                    /*liabilities*/
                    f.lct, f.dlc, f.dltt, f.lt, f.dm, f.dcvt, f.cshrc, 
                    f.dcpstk, f.pstk, f.ap, f.lco, f.lo, f.drc, f.drlt, f.txdi,

                    /*equity and other*/
                    f.ceq, f.scstkc, f.emp, f.csho, f.seq, f.txditc, f.pstkrv, f.pstkl, f.np, f.txdc, f.dpc, f.ajex,

                    /*market*/
                    abs(f.prcc_f) as prcc_f

                    from comp.funda as f
                    left join comp.company as c
                    on f.gvkey = c.gvkey

                    /*get consolidated, standardized, industrial format statements*/
                    where f.indfmt = 'INDL' 
                    and f.datafmt = 'STD'
                    and f.popsrc = 'D'
                    and f.consol = 'C'
                    and f.datadate >= '01/01/%(start_year)s'
                    and f.datadate <= '12/31/%(end_year)s'
                    """, params=parm)

comp_crsp_link = conn.raw_sql("""
                  select gvkey, lpermno as permno, linktype, linkprim, 
                  linkdt, linkenddt
                  from crsp.ccmxpf_linktable
                  where substr(linktype,1,1)='L'
                  and (linkprim ='C' or linkprim='P')
                  """)

conn.close()

# Merge permno with compustat data

comp_crsp_link.drop(comp_crsp_link[comp_crsp_link.linktype == "LD"].index, inplace=True)

comp_crsp_link['linkdt'] = pd.to_datetime(comp_crsp_link['linkdt'])
comp_crsp_link['linkenddt'] = pd.to_datetime(comp_crsp_link['linkenddt'])
comp_crsp_link['linkenddt'] = comp_crsp_link['linkenddt'].fillna(pd.to_datetime('today'))

raw_annual = comp_annual.merge(comp_crsp_link[['gvkey', 'permno', 'linkdt', 'linkenddt']], how='left', on=['gvkey'])
raw_annual['permno'] = raw_annual['permno'].astype('Int64')

raw_annual['datadate'] = pd.to_datetime(raw_annual['datadate'])
raw_annual = raw_annual.sort_values(["permno", "datadate"]).drop_duplicates()
raw_annual.dropna(subset=["fyear", "permno"], inplace=True)

# Set link date bounds
raw_annual = raw_annual[(raw_annual['datadate'] >= raw_annual['linkdt']) & (raw_annual['datadate'] <= raw_annual['linkenddt'])]
raw_annual = raw_annual.drop(['linkdt', 'linkenddt'], axis=1)

# Convert sic and sic2 columns from object to numeric data type
raw_annual['sic'] = pd.to_numeric(raw_annual['sic'])
raw_annual['sic2'] = pd.to_numeric(raw_annual['sic2'])

In [ ]:
month = crsp_msf.copy()
month["date"] = pd.to_datetime(month["date"])
month["date_std"] = month["date"] + MonthEnd(0)
month["date_std"] = pd.to_datetime(month["date_std"])
month["permno"] = month["permno"].astype('int64')

annual = raw_annual.copy()
annual["datadate"] = pd.to_datetime(annual["datadate"])
annual["date_std"] = annual["datadate"] + MonthEnd(0)

In [ ]:
combined = pd.merge(month, annual, how="left", on=["permno", "date_std"])
combined.sort_values(["permno", "date_std"], inplace=True)

In [ ]:
annual_cols = ['gvkey', 'cusip', 'datadate', 'fyear', 'cik', 'sic2', 'sic', 'naics',
       'sale', 'revt', 'cogs', 'xsga', 'dp', 'xrd', 'xad', 'ib', 'ebitda',
       'ebit', 'nopi', 'spi', 'pi', 'txp', 'ni', 'txfed', 'txfo', 'txt',
       'xint', 'capx', 'oancf', 'dvt', 'ob', 'gdwlia', 'gdwlip', 'gwo', 'mib',
       'oiadp', 'ivao', 'rect', 'act', 'che', 'ppegt', 'invt', 'at', 'aco',
       'intan', 'ao', 'ppent', 'gdwl', 'fatb', 'fatl', 'lct', 'dlc', 'dltt',
       'lt', 'dm', 'dcvt', 'cshrc', 'dcpstk', 'pstk', 'ap', 'lco', 'lo', 'drc',
       'drlt', 'txdi', 'ceq', 'scstkc', 'emp', 'csho', 'seq', 'txditc',
       'pstkrv', 'pstkl', 'np', 'txdc', 'dpc', 'ajex', 'prcc_f', 'permno',
       'date_std']

combined[annual_cols] = combined[annual_cols].ffill(axis=0, limit=12)

In [ ]:
combined

In [ ]:
# Calculate features

# Anomaly characteristics

#1 Size
size
size = MEJun

#2 Value (annual)
value
value = BE/ME

#3 Gross Profitability
prof
prof = GP/AT

#4 Value-Profitability
valprof
valprof = rank(value) + rank(prof)

#5 Piotroski’s F-score
F-score
F-score = 1IB>0 + 1ROA>0 +
1CFO>0 +1CFO>IB +1DTA<0|DLTT=0|DLTT−12=0 +1ATL>0 +1EqIss0 +1GM>0 +1ATO>0

#6 Debt Issuance
debtiss
debtiss = 1DLTISS0

#7 Share Repurchases
repurch
repurch = 1PRSTKC>0

#8 Share Issuance (annual)
nissa
nissa = shroutJun / shroutJun−12

#9 Accruals 
accruals
accruals = ACT−CHE−LCT+DLC+TXP−DP (AT+AT−12)/2

#10 Asset Growth
growth
growth = AT/AT−12.

#11 Asset Turnover
aturnover
aturnover = SALE/AT

#12 Gross Margins
gmargins
gmargins = GP/SALE

#13 Dividend Yield
divp
divp = Div/MEDec

#14 Earnings/Price
ep
ep = IB/MEDec

#15 Cash Flow / Market Value of Equity
cfp
cfp = (IB + DP)/MEDec

#16 Net Operating Assets
noa
noa = (AT - CHE) - (AT - DLC - DLTT - MIB - PSTK - CEQ)

#17 Investment
inv
inv = (PPEGT + INVT)/AT−12

#18 Investment-to-Capital
invcap
invcap = CAPX/PPENT

#19 Investment Growth
growth
growth = CAPX/CAPX−12

#20 Sales Growth
sgrowth
sgrowth = SALE/SALE−12

#21 Leverage
lev
lev = AT/MEDec

#22 Return on Assets (annual)
roaa
roaa = IB/AT

#23 Return on Equity (annual)
roea
roea = IB/BE

#24 Sales-to-Price
sp
sp = SALE/MEDec

#25 Growth in LTNOA
gltnoa
NOA = (RECT + INVT + ACO + PPENT
+ INTAN + AO - AP - LCO - LO) / AT, GRNOA = NOA - NOA−12, ACC=((RECT -
RECT−12) + (INVT - INVT−12) + (ACO - ACO−12) - (AP - AP−12) - (LCO - LCO−12) - DP)
/ ((AT + AT−12) / 2)

#26 Momentum (6m)
mom
mom = P7l =2 rt−l

#27 Industry Momentum
indmom
indmom = rank(P6l =1 rind t−l).

#28 Value-Momentum
valmom
valmom = rank(B/M) + rank(Mom)

#29 Value-Momentum-Profitability
valmomprof
valmomprof = rank(B/M) + rank(Prof) + rank(Mom)

#30 Short Interest
shortint
shortint = Shares Shorted / Shares Outstanding

#31 Momentum (1 year) 
mom12
mom12 =P12 l=2 rt−l

#32 Momentum-Reversal
momrev
momrev = P19 l=14 rt−l

#33 Long-term Reversals
lrrev
lrrev = P60 l=13 rt−l.

#34 Value (monthly)
valuem
valuem = BEQ−3/ME−1

#35 Share Issuance (monthly)
nissm
nissm = shroutt−13 / shroutt−1

#36 PEAD (SUE)
sue
sue = IBQ−IBQ−12
IBQ−24:IBQ−3

#37 Return on Book Equity
roe
roe = IBQ/BEQ−3

#38 Return on Market Equity
rome
rome = IBQ/ME−4

#39 Return on Assets 
roa
roa = IBQ/ATQ−3

#40 Short-term Reversal 
strev
strev = rt−1

#41 Idiosyncratic Volatility 
ivol
ivol = std(Ri,t − iRM,t − siSMBt−hiHMLt)

#42 Beta Arbitrage 
beta
beta = t−60:t−1
    
#43 Seasonality
season
season = P5l = 1 rt−l×12

#44 Industry Relative Reversals
indrrev
indrrev = r−1 − rind −1

#45 Industry Relative Reversals (Low Volatility)
indrrevlv
indrrevlv = r−1 − rind −1 if vol < NYSE

#46 Industry Momentum-Reversal
indmomrev
indmomrev = rank(industry momentum) + rank(industry relative-reversals low-vol)

#47 Composite Issuance
ciss
ciss = log(MEt−13 MEt−60 ) − P60 l=13 rt−l

#48 Price
price
price = log(ME/shrout
            
#49 Firm Age
age
age = log(1 + number of months since listing)
            
#50 Share Volume
shvol
shvol = 13 P3i = 1 volumet−i/shroutt.

In [ ]:
# Calculate features

# WRDS financial ratios

1. P/E (Diluted, Excl. EI) (pe_exi) – Valuation. Price-to-Earnings, excl. Extraordinary
Items (diluted).

2. P/E (Diluted, Incl. EI) (pe_inc) – Valuation. Price-to-Earnings, incl. Extraordinary
Items (diluted).

3. Price/Sales (ps) – Valuation. Multiple of Market Value of Equity to Sales.

4. Price/Cash flow (pcf ) – Valuation. Multiple of Market Value of Equity to Net Cash Flow
from Operating Activities.

5. Enterprise Value Multiple (evm) – Valuation. Multiple of Enterprise Value to EBITDA.

6. Book/Market (bm) – Valuation. Book Value of Equity as a fraction of Market Value of
Equity.

7. Shiller’s Cyclically Adjusted P/E Ratio (capei) – Valuation. Multiple of Market Value
of Equity to 5-year moving average of Net Income.

8. Dividend Payout Ratio (dpr) – Valuation. Dividends as a fraction of Income Before Extra.
Items.

9. Net Profit Margin (npm) – Profitability. Net Income as a fraction of Sales.

10. Operating Profit Margin Before Depreciation (opmbd) – Profitability. Operating Income
Before Depreciation as a fraction of Sales.

11. Operating Profit Margin After Depreciation (opmad) – Profitability. Operating Income
After Depreciation as a fraction of Sales.

12. Gross Profit Margin (gpm) – Profitability. Gross Profit as a fraction of Sales.

13. Pre-tax Profit Margin (ptpm) – Profitability. Pretax Income as a fraction of Sales.

14. Cash Flow Margin (cfm) – Financial Soundness. Income before Extraordinary Items and
Depreciation as a fraction of Sales.

15. Return on Assets (roa) – Profitability. Operating Income Before Depreciation as a fraction
of average Total Assets based on most recent two periods.

16. Return on Equity (roe) – Profitability. Net Income as a fraction of average Book Equity
based on most recent two periods, where Book Equity is defined as the sum of Total Parent
Stockholders’ Equity and Deferred Taxes and Investment Tax Credit.

17. Return on Capital Employed (roce) – Profitability. Earnings Before Interest and Taxes
as a fraction of average Capital Employed based on most recent two periods, where Capital
Employed is the sum of Debt in Long-term and Current Liabilities and Common/Ordinary
Equity.

18. After-tax Return on Average Common Equity (aftret_eq) – Profitability. Net Income
as a fraction of average of Common Equity based on most recent two periods.

19. After-tax Return on Invested Capital (aftret_invcapx) – Profitability. Net Income plus
Interest Expenses as a fraction of Invested Capital.

20. After-tax Return on Total Stockholders’ Equity (aftret_equity) – Profitability. Net
Income as a fraction of average of Total Shareholders’ Equity based on most recent two
periods.

21. Pre-tax return on Net Operating Assets (pretret_noa) – Profitability. Operating Income
After Depreciation as a fraction of average Net Operating Assets (NOA) based on most
recent two periods, where NOA is defined as the sum of Property Plant and Equipment and
Current Assets minus Current Liabilities.

22. Pre-tax Return on Total Earning Assets (pretret_earnat) – Profitability. Operating
Income After Depreciation as a fraction of average Total Earnings Assets (TEA) based on
most recent two periods, where TEA is defined as the sum of Property Plant and Equipment
and Current Assets.

23. Common Equity/Invested Capital (equity_invcap) – Capitalization. Common Equity
as a fraction of Invested Capital.

24. Long-term Debt/Invested Capital (debt_invcap) – Capitalization. Long-term Debt as
a fraction of Invested Capital.

25. Total Debt/Invested Capital (totdebt_invcap) – Capitalization. Total Debt (Long-term
and Current) as a fraction of Invested Capital.

26. Interest/Average Long-term Debt (int_debt) – Financial Soundness. Interest as a fraction
of average Long-term debt based on most recent two periods.

27. Interest/Average Total Debt (int_totdebt) – Financial Soundness. Interest as a fraction
of average Total Debt based on most recent two periods.

28. Cash Balance/Total Liabilities (cash_lt) – Financial Soundness. Cash Balance as a
fraction of Total Liabilities.

29. Inventory/Current Assets (invt_act) – Financial Soundness. Inventories as a fraction of
Current Assets.

30. Receivables/Current Assets (rect_act) – Financial Soundness. Accounts Receivables as
a fraction of Current Assets.

31. Total Debt/Total Assets (debt_at) – Solvency. Total Liabilities as a fraction of Total
Assets.

32. Short-Term Debt/Total Debt (short_debt) – Financial Soundness. Short-term Debt as
a fraction of Total Debt.

33. Current Liabilities/Total Liabilities (curr_debt) – Financial Soundness. Current Liabilities
as a fraction of Total Liabilities.

34. Long-term Debt/Total Liabilities (lt_debt) – Financial Soundness. Long-term Debt as
a fraction of Total Liabilities.

35. Free Cash Flow/Operating Cash Flow (fcf_ocf ) – Financial Soundness. Free Cash
Flow as a fraction of Operating Cash Flow, where Free Cash Flow is defined as the difference
between Operating Cash Flow and Capital Expenditures.

36. Avertising Expenses/Sales (adv_sale) – Other. Advertising Expenses as a fraction of
Sales.

37. Profit Before Depreciation/Current Liabilities (profit_lct) – Financial Soundness. Operating
Income before D&A as a fraction of Current Liabilities.

38. Total Debt/EBITDA (debt_ebitda) – Financial Soundness. Gross Debt as a fraction of
EBITDA.

39. Operating CF/Current Liabilities (ocf_lct) – Financial Soundness. Operating Cash
Flow as a fraction of Current Liabilities.

40. Total Liabilities/Total Tangible Assets (lt_ppent) – Financial Soundness. Total Liabilities
to Total Tangible Assets.

41. Long-term Debt/Book Equity (dltt_be) – Financial Soundness. Long-term Debt to Book
Equity.

42. Total Debt/Total Assets (debt_assets) – Solvency. Total Debt as a fraction of Total
Assets.

43. Total Debt/Capital (debt_capital) – Solvency. Total Debt as a fraction of Total Capital,
where Total Debt is defined as the sum of Accounts Payable and Total Debt in Current and
Long-term Liabilities, and Total Capital is defined as the sum of Total Debt and Total Equity
(common and preferred).

44. Total Debt/Equity (de_ratio) – Solvency. Total Liabilities to Shareholders’ Equity (common
and preferred).

45. After-tax Interest Coverage (intcov) – Solvency. Multiple of After-tax Income to Interest
and Related Expenses.

46. Cash Ratio (cash_ratio) – Liquidity. Cash and Short-term Investments as a fraction of
Current Liabilities.

47. Quick Ratio (Acid Test) (quick_ratio) – Liquidity. Quick Ratio: Current Assets net of
Inventories as a fraction of Current Liabilities.

48. Current Ratio (curr_ratio) – Liquidity. Current Assets as a fraction of Current Liabilities.

49. Capitalization Ratio (capital_ratio) – Capitalization. Total Long-term Debt as a fraction
of the sum of Total Long-term Debt, Common/Ordinary Equity and Preferred Stock.

50. Cash Flow/Total Debt (cash_debt) – Financial Soundness. Operating Cash Flow as a
fraction of Total Debt.

51. Inventory Turnover (inv_turn) – Efficiency. COGS as a fraction of the average Inventories
based on the most recent two periods.

52. Asset Turnover (at_turn) – Efficiency. Sales as a fraction of the average Total Assets based
on the most recent two periods.

53. Receivables Turnover (rect_turn) – Efficiency. Sales as a fraction of the average of Accounts
Receivables based on the most recent two periods.

54. Payables Turnover (pay_turn) – Efficiency. COGS and change in Inventories as a fraction
of the average of Accounts Payable based on the most recent two periods.

55. Sales/Invested Capital (sale_invcap) – Efficiency. Sales per dollar of Invested Capital.

56. Sales/Stockholders Equity (sale_equity) – Efficiency. Sales per dollar of total Stockholders’
Equity.

57. Sales/Working Capital (sale_nwc) – Efficiency. Sales per dollar of Working Capital,
defined as difference between Current Assets and Current Liabilities.

58. Research and Development/Sales (RD_SALE) – Other. R&D expenses as a fraction of
Sales.

59. Accruals/Average Assets (Accrual) – Other. Accruals as a fraction of average Total Assets
based on most recent two periods.

60. Gross Profit/Total Assets (GProf ) – Profitability. Gross Profitability as a fraction of
Total Assets.

61. Book Equity (be) – Other. Firm size as measured by total book equity.

62. Cash Conversion Cycle (Days) (cash_conversion) – Liquidity. Inventories per daily
COGS plus Account Receivables per daily Sales minus Account Payables per daily COGS.

63. Effective Tax Rate (efftax) – Profitability. Income Tax as a fraction of Pretax Income.

64. Interest Coverage Ratio (intcov_ratio) – Solvency. Multiple of Earnings Before Interest
and Taxes to Interest and Related Expenses.

65. Labor Expenses/Sales (staff_sale) – Other. Labor Expenses as a fraction of Sales.

66. Dividend Yield (divyield) – Valuation. Indicated Dividend Rate as a fraction of Price.

67. Price/Book (ptb) – Valuation. Multiple of Market Value of Equity to Book Value of Equity.

68. Trailing P/E to Growth (PEG) ratio (PEG_trailing) – Valuation. Price-to-Earnings,
excl. Extraordinary Items (diluted) to 3-Year past EPS Growth.

69-80. Return in Month t − i (ret_lagi) – Other. Past one-month returns in months t − i for
i = {1, ..., 12}.